# Janomeを使った単語化

以下手順を順に行っていきます。

- 前週に取得した一橋大学のニュース文章を利用
- 単語化
- 名詞のみを取得
- 不要な単語を設定
- 文字をノーマライズ
- 単語の登場頻度Top10を出力

## ニュースの文字をファイルから取得

In [1]:
# ColabでPathがうまく取れない場合は次のセルを参照
with open("20210502-news-text.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [2]:
# Colabでファイルが取得できない場合
import csv
import urllib.request
import base64
import io
url = "https://hit-u-data-text-processing.herokuapp.com/data/20210502-news-text.txt"
auth_str = base64.b64encode(b"reader:hit-u")
req = urllib.request.Request(url,
                            headers={"Authorization": "Basic " + auth_str.decode("utf-8")})

links = []
with urllib.request.urlopen(req) as req:
    text = req.read().decode("utf-8")

In [3]:
len(text)

14914

In [4]:
text[:200]

'\n一橋大学がソニー、パナソニック、富士通、資生堂と共同で、 デザイン組織の共通評価指標を検討・作成\n2021年4月30日\n\n 一橋大学は、経営管理研究科内に設置した「データ・デザイン研究センター」において、ソニーグループ株式会社クリエイティブセンター、パナソニック株式会社デザイン本部、富士通株式会社デザインセンター及び株式会社資生堂クリエイティブ本部と共同で、社内デザイン組織の活動や成果を共通の視'

## 単語化

In [5]:
from janome.tokenizer import Tokenizer

In [6]:
t = Tokenizer()
words = list(t.tokenize(text, wakati=True))

In [7]:
len(words)

7651

In [8]:
words[:10]

['一橋大学', 'が', 'ソニー', '、', 'パナソニック', '、', '富士通', '、', '資生堂', 'と']

## Janomeにフィルターなどを設定する

In [9]:
from janome.tokenfilter import TokenFilter
from janome.tokenfilter import CompoundNounFilter
from janome.tokenfilter import POSKeepFilter
from janome.tokenfilter import LowerCaseFilter
from janome.charfilter import UnicodeNormalizeCharFilter
from janome.analyzer import Analyzer

In [10]:
class StopWordFilter(TokenFilter):
    def __init__(self, words):
        self.stop_words = words
    
    def apply(self, tokens):
        for token in tokens:
            if token.surface not in self.stop_words:
                yield token

In [11]:
stop_word_filter = StopWordFilter(["大学", "一橋大学"])

In [12]:
token_filters = [CompoundNounFilter(),
                POSKeepFilter(["名詞"]),
                LowerCaseFilter(),
                stop_word_filter]

In [13]:
char_filters = [UnicodeNormalizeCharFilter()]

In [14]:
tokenizer = Tokenizer()

In [15]:
analyzer = Analyzer(char_filters=char_filters, 
                    tokenizer=tokenizer, 
                    token_filters=token_filters)

In [16]:
noun_words = []
for token in analyzer.analyze(text):
    noun_words.append(token.base_form)

In [17]:
len(noun_words)

1841

In [18]:
noun_words[:10]

['ソニー',
 'パナソニック',
 '富士通',
 '資生堂',
 '共同',
 'デザイン組織',
 '共通評価指標',
 '検討',
 '作成',
 '2021年4月30日']

## 登場頻度Top10

第４回で使った `from collections import Counter` を使う

In [19]:
from collections import Counter

In [20]:
c = Counter(noun_words)

In [21]:
c.most_common(10)

[('===', 18),
 ('こと', 14),
 ('the', 14),
 ('ため', 13),
 ('よう', 12),
 ('参加', 12),
 ('世界', 12),
 ('結果', 11),
 ('デザイン組織', 10),
 ('裁判員裁判', 10)]